In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import pickle
from scipy import spatial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy import random

# data preperation

In [2]:
data = pd.read_csv("./bank_data/bank-full.csv", sep = ";")
data["weight"] = 1
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,weight
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,1
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,1


In [3]:
train_data, test_data = train_test_split(data, stratify = data["y"], 
                                         train_size=0.01)
train_data = train_data.reset_index(drop=True).copy()
test_data = test_data.reset_index(drop=True).copy()
train_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,weight
0,44,admin.,married,secondary,no,14,no,yes,unknown,30,may,827,5,-1,0,unknown,no,1
1,52,housemaid,single,tertiary,no,0,no,no,unknown,6,jun,431,2,-1,0,unknown,no,1
2,32,admin.,divorced,secondary,no,1288,yes,no,cellular,12,may,286,1,-1,0,unknown,no,1
3,34,technician,married,secondary,no,173,no,no,unknown,4,jul,31,1,-1,0,unknown,no,1
4,59,housemaid,divorced,secondary,no,122,no,no,cellular,15,jul,129,1,-1,0,unknown,no,1


In [4]:
len(train_data)

452

In [5]:
len(test_data)

44759

In [6]:
# A list of the numerical feature names.
NUMERIC_FEATURE_NAMES = [
    "age",
    "balance",
    "duration",
    "campaign",
    "previous",
    "pdays",
]

CATEGORICAL_FEATURE_NAMES = [
    "job",
    "marital",
    "education",
    "default",
    "housing",
    "loan",
    "contact",
    "day",
    "month",
    "poutcome"
]

WEIGHT_COLUMN_NAME = "weight"

# The name of the target feature.
TARGET_FEATURE_NAME = "y"
# A list of the labels of the target features.
TARGET_LABELS = ["no", "yes"]

# Embedding Codes

In [7]:
def save_embeddings(model, filename):
    embeddings = {}

    for layer in model.layers: 
        if "_embedding" in  layer.get_config()["name"]:
            col_name = layer.get_config()["name"].split("_embedding")[0]
            if col_name not in CATEGORICAL_FEATURES_WITH_VOCABULARY:
                continue
            embeddings[col_name] = {}
            for idx, cat in enumerate(CATEGORICAL_FEATURES_WITH_VOCABULARY[col_name]):
                if "mask" in cat:
                    continue
                embeddings[col_name][cat] = layer.get_weights()[0][idx]
            
    with open(filename, 'wb') as config_dictionary_file:
        pickle.dump(embeddings, config_dictionary_file)

In [8]:
def load_embeddings(model, filename):
    with open(filename, 'rb') as config_dictionary_file:
        embeddings = pickle.load(config_dictionary_file)
    
    for layer in model.layers: 
        if "_embedding" in  layer.get_config()["name"]:
            col_name = layer.get_config()["name"].split("_embedding")[0]
            if col_name not in CATEGORICAL_FEATURES_WITH_VOCABULARY:
                continue
                
            layer.set_weights([np.array(list(
                        [
                            embeddings[col_name][c] \
                                             for c in CATEGORICAL_FEATURES_WITH_VOCABULARY[col_name]
                        ]
            ))])
    return model

In [9]:
train_data.sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,weight
145,28,blue-collar,single,secondary,no,97,yes,no,unknown,7,may,67,1,-1,0,unknown,no,1
386,22,blue-collar,single,secondary,no,0,yes,no,unknown,19,may,123,1,-1,0,unknown,no,1
74,52,services,married,secondary,no,393,yes,yes,cellular,28,jan,446,1,182,8,failure,no,1
215,40,management,divorced,tertiary,no,858,no,no,cellular,26,oct,202,2,-1,0,unknown,yes,1
367,32,management,single,tertiary,no,223,no,no,cellular,29,jan,365,1,-1,0,unknown,no,1


In [10]:
test_data.sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,weight
27490,47,blue-collar,married,primary,no,9346,no,no,cellular,22,aug,19,21,-1,0,unknown,no,1
33175,44,admin.,married,secondary,no,73,yes,no,unknown,16,may,404,1,-1,0,unknown,no,1
40433,36,blue-collar,married,secondary,no,0,yes,no,cellular,7,may,332,1,-1,0,unknown,no,1
3139,48,technician,divorced,tertiary,no,2328,no,no,cellular,27,aug,129,7,-1,0,unknown,no,1
2036,37,technician,single,tertiary,no,178,yes,no,unknown,21,may,97,1,-1,0,unknown,no,1


In [11]:
CSV_HEADER = data.columns

# A list of all the input features.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME] else ["NA"]
    for feature_name in CSV_HEADER
]

CATEGORICAL_FEATURES_WITH_VOCABULARY = {}

for f in CATEGORICAL_FEATURE_NAMES:
    
    train_data[f] = train_data[f].astype("str")
    test_data[f] = test_data[f].astype("str")
    
    CATEGORICAL_FEATURES_WITH_VOCABULARY[f] =\
                    sorted(list(train_data[f].unique()))

# The name of the target feature.
TARGET_FEATURE_NAME = "y"
# A list of the labels of the target features.
TARGET_LABELS = train_data[TARGET_FEATURE_NAME].unique()

In [12]:
train_data_file = "train_data.csv"
test_data_file = "test_data.csv"

train_data.to_csv(train_data_file, index=False, header=True)
test_data.to_csv(test_data_file, index=False, header=True)

In [13]:
"""
## Configure the hyperparameters
The hyperparameters includes model architecture and training configurations.
"""

LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 128
NUM_EPOCHS = 10

NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 8  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

"""
## Implement data reading pipeline
We define an input function that reads and parses the file, then converts features
and labels into a[`tf.data.Dataset`](https://www.tensorflow.org/guide/datasets)
for training or evaluation.
"""

target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)

def prepare_example(features, target):
    target_index = target_label_lookup(target)
    weights = features.pop(WEIGHT_COLUMN_NAME)
    return features, target_index, weights


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        #column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=True,
        na_value="?",
        shuffle=shuffle,
    ).map(prepare_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    return dataset.cache()

"""
## Create model inputs
Now, define the inputs for the models as a dictionary, where the key is the feature name,
and the value is a `keras.layers.Input` tensor with the corresponding feature shape
and data type.
"""


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs


"""
## Encode features
The `encode_inputs` method returns `encoded_categorical_feature_list` and `numerical_feature_list`.
We encode the categorical features as embeddings, using a fixed `embedding_dims` for all the features,
regardless their vocabulary sizes. This is required for the Transformer model.
"""


def encode_inputs(inputs, embedding_dims):

    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:

            # Get the vocabulary of the categorical feature.
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]

            # Create a lookup to convert string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = layers.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=1,
                output_mode="int",
                name = feature_name+"_string_lookup"
            )

            # Convert the string input values into integer indices.
            encoded_feature = lookup(inputs[feature_name])-1

            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims,
                name=feature_name+"_embedding"
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(encoded_feature)
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:

            # Use the numerical features as-is.
            numerical_feature = tf.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list


"""
## Implement an MLP block
"""


def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):

    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer),
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)

"""
## Implement a training and evaluation procedure
"""


def run_experiment(
    model,
    train_data_file,
    test_data_file,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

    print("Start training the model...")
    history = model.fit(
        train_dataset, epochs=num_epochs, validation_data=validation_dataset
    )
    print("Model training finished")

    _, accuracy = model.evaluate(validation_dataset, verbose=0)

    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")

    return history, model

In [14]:
def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    use_column_embedding=False,
):

    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Stack categorical feature embeddings for the Tansformer.
    encoded_categorical_features = tf.stack(encoded_categorical_feature_list, axis=1)
    # Concatenate numerical features.
    numerical_features = layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        column_indices = tf.range(start=0, limit=num_columns, delta=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)
        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=layers.LayerNormalization(epsilon=1e-6),
            name=f"feedforward_{block_idx}",
        )(x)
        # Skip connection 2.
        x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
        # Layer normalization 2.
        encoded_categorical_features = layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)

    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = layers.Flatten(name="dyanmic_embedding")(encoded_categorical_features)
    # Apply layer normalization to the numerical features.
    numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
    # Prepare the input for the final MLP block.
    features = layers.concatenate([categorical_features, numerical_features])

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization(),
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", tabtransformer_model.count_params())
keras.utils.plot_model(tabtransformer_model, show_shapes=True, rankdir="LR")

"""
Let's train and evaluate the TabTransformer model:
"""

tabtransformer_model = load_embeddings(tabtransformer_model, 
                                       'untrained_embeddings_bank.dictionary')

history, tabtransformer_model = run_experiment(
    model=tabtransformer_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Total model weights: 34629
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Start training the model...
Epoch 1/10
      1/Unknown - 4s 4s/step - loss: 1.0878 - accuracy: 0.4609WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.
4/4 [==============================] - 8s 1s/step - loss: 0.8759 - accuracy: 0.5310 - val_loss: 0.6033 - val_accuracy: 0.6854
Epoch 2/10
4/4 [==============================] - 2s 557ms/step - loss: 0.6678 - accuracy: 0.6593 - val_loss: 0.4604 - val_accuracy: 0.8388
Epoch 3/10
4/4 [==============================] - 2s 541ms/step - loss: 0.5178 - accuracy: 0.7345 - val_loss: 0.4152 - val_accuracy: 0.8729
Epoch 4/10
4/4 [=============

In [15]:
save_embeddings(tabtransformer_model,
                'supervised_trained_embeddings_bank.dictionary')